In [1]:
import os
import numpy as np
import pandas as pd


import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

In [2]:
train_dir = '../data/seg_train/seg_train'
test_dir = '../data/seg_test/seg_test'
pred_dir = '../data/seg_pred/seg_pred'

# implement data augmentation
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip=True
                )

test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = train_datagen.flow_from_directory(
                        train_dir,
                        target_size=(150,150),
                        batch_size = 140,
                        class_mode='categorical'
                    )

Found 14034 images belonging to 6 classes.


In [4]:
val_generator = test_datagen.flow_from_directory(
                    test_dir,
                    target_size = (150,150),
                    batch_size = 30,    
                    class_mode = 'categorical'
                )

Found 3000 images belonging to 6 classes.


In [5]:
# build Conv Net with regularization

## instantiate basic Sequential model
model = keras.Sequential()

# add 1st Conv layer (w/ Max Pooling)
model.add( Conv2D(filters = 64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(150,150,3)) )
model.add( Conv2D(filters = 64, kernel_size=(3,3), activation='relu', padding='same') )
model.add( MaxPooling2D( pool_size=(2,2) ) )

# add 2nd Conv layer (w/ Max Pooling)
model.add( Conv2D(filters = 128, kernel_size=(3,3), activation='relu', padding='same') )
model.add( Conv2D(filters = 128, kernel_size=(3,3), activation='relu') )
model.add( MaxPooling2D( pool_size=(2,2) ))

# add 3rd Conv layer (w/Max Pooling)
model.add( Conv2D(filters = 256, kernel_size=(3,3), activation='relu', padding='same') )
model.add( Conv2D(filters = 256, kernel_size=(3,3), activation='relu') )
model.add( MaxPooling2D( pool_size=(2,2) ))

# add 4th Conv layer (w/ Max Pooling)
model.add( Conv2D(filters = 512, kernel_size=(3,3), activation='relu', padding='same') )
model.add( Conv2D(filters = 512, kernel_size=(3,3), activation='relu') )
model.add( MaxPooling2D( pool_size=(2,2) ))

# flatten
model.add( Flatten())

# add drop out later for regularization
model.add( Dropout(0.5))

# hidden layers and output
model.add( Dense(units=1024, activation='relu') )
model.add( Dense(units=512, activation='relu') )
model.add( Dense(units=6, activation='softmax') )


model.compile(
        loss = 'categorical_crossentropy',
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005),
        metrics = ['acc']
)

In [6]:
# take a look at the architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 150, 150, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 75, 75, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 73, 73, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 128)      0         
 2D)                                                    

In [7]:
tboard = TensorBoard(log_dir='./logs', write_images=True, )
stopper = EarlyStopping(patience=5)
checkpts = ModelCheckpoint(filepath='./checkpoints/model-1/model-1', save_best_only=True, save_weights_only=True)



model.fit(
        train_generator,
        steps_per_epoch=100,
        epochs = 100,
        validation_data = val_generator,
        validation_steps = 100,  # this parameter is very important because our val data is a generator: if not specified, will loop forever!
        callbacks = [tboard, stopper, checkpts]
)

Epoch 1/100
100/100 [==============================] - 56s 494ms/step - loss: 1.3761 - acc: 0.4242 - val_loss: 1.2093 - val_acc: 0.5337
Epoch 2/100
100/100 [==============================] - 49s 491ms/step - loss: 1.0914 - acc: 0.5537 - val_loss: 0.9126 - val_acc: 0.6270
Epoch 3/100
100/100 [==============================] - 49s 487ms/step - loss: 0.9976 - acc: 0.5961 - val_loss: 0.8968 - val_acc: 0.6437
Epoch 4/100
100/100 [==============================] - 49s 494ms/step - loss: 0.9272 - acc: 0.6424 - val_loss: 0.8514 - val_acc: 0.6823
Epoch 5/100
100/100 [==============================] - 50s 494ms/step - loss: 0.8302 - acc: 0.6873 - val_loss: 0.6542 - val_acc: 0.7593
Epoch 6/100
100/100 [==============================] - 49s 493ms/step - loss: 0.7731 - acc: 0.7130 - val_loss: 0.5751 - val_acc: 0.7957
Epoch 7/100
100/100 [==============================] - 49s 484ms/step - loss: 0.7247 - acc: 0.7350 - val_loss: 0.6178 - val_acc: 0.7760
Epoch 8/100
100/100 [===========================

In [ ]:
model.save('../models/model-2')